#Information
Group 9 : Resume parser

Content : Similarity of resumes and job description using Bert

In [1]:
# packages
import pandas as pd
import numpy as np
import seaborn as sns
import re
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import  word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Connect to drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Move the directory to the csv files
%cd /content/drive/MyDrive/NLP_Final_changes/

/content/drive/MyDrive/NLP_Final_changes


In [ ]:
ls

Bert_classification.ipynb                 sample_java_job_description.docx
Bert_Similarity.ipynb                     sample_java_resumes.csv
Category_plot.png                         sample_testing_job_description.docx
cleaned_resume_final.csv                  sample_testing_resumes.csv
Preprocessing.ipynb                       TF_IDF_Classification.ipynb
resume_classification_model/              TF_IDF_Similarity.ipynb
ResumeDataSet.csv                         Word2Vec.ipynb
sample_data_science_job_description.docx  Word_cloud.png
sample_data_science_resumes.csv


#Sample data science resumes and job description

In [5]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_data_science_resumes.csv")

In [6]:
# Same preprocessing function used while preprocessing the resumes
# Preprocessing the data
def preprocess_text(sentence_list):
  lines_list = []
  stop_words = set(stopwords.words('english'))
  lemmatizer = WordNetLemmatizer()

  for line in sentence_list:
      line = line.lower()
      words=[]
      for word in word_tokenize(line):
        if word.isalpha():
          if word not in stop_words:
            words.append(lemmatizer.lemmatize(word))
      lines_list.append(' '.join(words))
  
  return lines_list

In [7]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [8]:
df.head()

,Category,Resume,cleaned_resume
0,Data Science,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Data Science,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Data Science,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Data Science,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [9]:
!pip install docx2txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3977 sha256=0fa87ca11bd818370c09f027d5a89ed4144aefb88002a083c3420db24c8b20c7
  Stored in directory: /root/.cache/pip/wheels/40/75/01/e6c444034338bde9c7947d3467807f889123465c2371e77418
Successfully built docx2txt


In [10]:
import docx2txt

In [11]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_data_science_job_description.docx")

In [12]:
# Preprocess job description
job_description_tokenised = preprocess_text([job_description])

In [13]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 18.0 MB/s eta 0:00:00


In [14]:
import torch
from transformers import BertTokenizer, BertModel

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [92]:
# Whenever input tokens are passed, it returns the input embeddings
def get_embeddings(tokens):
  data_embeddings = []
  for i in range(len(tokens['input_ids'])):
      input_ids = tokens['input_ids'][i]
      attention_mask = tokens['attention_mask'][i]
      outputs = model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))
      data_embedding = outputs[1].detach().numpy()
      data_embeddings.append(data_embedding)
  
  return data_embeddings

In [93]:
# Generate tokens for job description
job_tokens = tokenizer.batch_encode_plus(job_description_tokenised, padding=True, truncation=True, max_length=256, return_tensors="pt")
# Generate embeddings for the job tokens
job_embedding = get_embeddings(job_tokens)[0]

In [95]:
# Generate tokens for the given list of resumes
resume_tokens = tokenizer.batch_encode_plus(df['cleaned_resume'], add_special_tokens=True, max_length=256, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')
# Generate embeddings for the list of resumes
resume_embeddings = get_embeddings(resume_tokens)

In [99]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# reshape the job and resume embeddings to get cosine similarity scores
job_embedding = job_embedding.reshape(job_embedding.shape[0], -1)
resume_embeddings = np.concatenate(resume_embeddings, axis=0)
resume_embeddings = resume_embeddings.reshape(resume_embeddings.shape[0], -1)

similarity_scores = cosine_similarity(job_embedding, resume_embeddings)

In [103]:
# Save cosine similarity to the dataset
df['Similarity_Percent'] = [round(i.item()*100, 2) for i in similarity_scores[0]]

In [104]:
df.head()

,Category,Resume,cleaned_resume,Similarity_Percent
0,Data Science,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,98.36
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,98.68
2,Data Science,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,97.96
3,Data Science,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,97.86
4,Data Science,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...,92.98


In [105]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [106]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [107]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
18,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skill ability quickly grasp technical...,98.81,1
10,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,98.79,2
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,98.68,3
16,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,98.48,4
0,Data Science,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,98.36,5


In [108]:
import random

In [109]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [110]:
# Shuffle the list
random.shuffle(manual_rank)

In [111]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
18,Data Science,Personal Skills â¢ Ability to quickly grasp t...,personal skill ability quickly grasp technical...,98.81,1,9
10,Data Science,Skills * Programming Languages: Python (pandas...,skill programming language python panda numpy ...,98.79,2,1
1,Data Science,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,98.68,3,13
16,Data Science,Skills â¢ Python â¢ Tableau â¢ Data Visuali...,skill python tableau data visualization r stud...,98.48,4,6
0,Data Science,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,98.36,5,14
17,Data Science,Education Details \r\n B.Tech Rayat and Bahr...,education detail rayat bahra institute enginee...,98.30,6,12
2,Data Science,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,97.96,7,17
3,Data Science,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,97.86,8,8
14,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",education detail mca ymcaust faridabad haryana...,97.53,9,18
7,Data Science,VIGNAN VENNAMPALLY vennampally.v@northeastern....,vignan vennampally linkedin github education n...,97.38,10,11


In [112]:
# Install rbo to use the rbo ranking similarity
!pip install rbo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [113]:
import rbo

In [114]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.550249179999567

#Sample java resumes and job description

In [118]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_java_resumes.csv")

In [119]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [120]:
df.head()

,Category,Resume,cleaned_resume
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Java Developer,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Java Developer,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Java Developer,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [121]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_java_job_description.docx")

In [122]:
# Preprocess job description
job_description_tokenised = preprocess_text([job_description])

In [123]:
# Generate tokens for job description
job_tokens = tokenizer.batch_encode_plus(job_description_tokenised, padding=True, truncation=True, max_length=256, return_tensors="pt")
# Generate embeddings for the job tokens
job_embedding = get_embeddings(job_tokens)[0]

In [124]:
# Generate tokens for the given list of resumes
resume_tokens = tokenizer.batch_encode_plus(df['cleaned_resume'], add_special_tokens=True, max_length=256, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')
# Generate embeddings for the list of resumes
resume_embeddings = get_embeddings(resume_tokens)

In [125]:
# reshape the job and resume embeddings to get cosine similarity scores
job_embedding = job_embedding.reshape(job_embedding.shape[0], -1)
resume_embeddings = np.concatenate(resume_embeddings, axis=0)
resume_embeddings = resume_embeddings.reshape(resume_embeddings.shape[0], -1)

similarity_scores = cosine_similarity(job_embedding, resume_embeddings)

In [126]:
# Save cosine similarity to the dataset
df['Similarity_Percent'] = [round(i.item()*100, 2) for i in similarity_scores[0]]

In [127]:
df.head()

,Category,Resume,cleaned_resume,Similarity_Percent
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,98.61
1,Java Developer,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,98.10
2,Java Developer,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,98.40
3,Java Developer,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,98.46
4,Java Developer,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...,94.66


In [128]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [129]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [130]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
11,Java Developer,"TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,...",technical skill skill java sql c bootstrap jsp...,99.28,1
13,Java Developer,"Operating Systems Windows XP, 7, 10. Tools/Pac...",operating system window xp visual basic uml pa...,99.08,2
10,Java Developer,Technical Skills (Trained / Project Acquired S...,technical skill trained project acquired skill...,98.85,3
20,Java Developer,Technical Skills (Trained / Project Acquired S...,technical skill trained project acquired skill...,98.85,4
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,98.61,5


In [131]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [132]:
# Shuffle the list
random.shuffle(manual_rank)

In [133]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
11,Java Developer,"TECHNICAL SKILLS Skills: Java, SQL, PL/SQL, C,...",technical skill skill java sql c bootstrap jsp...,99.28,1,6
13,Java Developer,"Operating Systems Windows XP, 7, 10. Tools/Pac...",operating system window xp visual basic uml pa...,99.08,2,13
10,Java Developer,Technical Skills (Trained / Project Acquired S...,technical skill trained project acquired skill...,98.85,3,17
20,Java Developer,Technical Skills (Trained / Project Acquired S...,technical skill trained project acquired skill...,98.85,4,15
0,Java Developer,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,98.61,5,2
7,Java Developer,VIGNAN VENNAMPALLY vennampally.v@northeastern....,vignan vennampally linkedin github education n...,98.48,6,21
3,Java Developer,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,98.46,7,16
2,Java Developer,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,98.40,8,8
16,Java Developer,TECHNICAL SKILLS Programming Languages: Java (...,technical skill programming language java serv...,98.38,9,9
6,Java Developer,"Nikitha Kanaparthi\nBoston, MA. 02120 • +1 (20...",nikitha kanaparthi boston linkedin github educ...,98.29,10,20


In [134]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.47439858820884906

#Sample testing resumes and job description

In [135]:
# Load required CSV files
# We have taken a sample of data science resumes to compare with a sample data science job description.
# Note: You can use your own category based on your requirement of job description

df=pd.read_csv("sample_testing_resumes.csv")

In [136]:
# add the cleaned text into the new column to save resources
df['cleaned_resume'] = preprocess_text(df['Resume'])

In [137]:
df.head()

,Category,Resume,cleaned_resume
0,Testing,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...
1,Testing,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...
2,Testing,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...
3,Testing,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...
4,Testing,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...


In [139]:
# Job description
# Get a sample job description
# Note: You can use your own job description based on requirement

job_description = docx2txt.process("sample_testing_job_description.docx")

In [140]:
# Preprocess job description
job_description_tokenised = preprocess_text([job_description])

In [141]:
# Generate tokens for job description
job_tokens = tokenizer.batch_encode_plus(job_description_tokenised, padding=True, truncation=True, max_length=256, return_tensors="pt")
# Generate embeddings for the job tokens
job_embedding = get_embeddings(job_tokens)[0]

In [142]:
# Generate tokens for the given list of resumes
resume_tokens = tokenizer.batch_encode_plus(df['cleaned_resume'], add_special_tokens=True, max_length=256, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt')
# Generate embeddings for the list of resumes
resume_embeddings = get_embeddings(resume_tokens)

In [143]:
# reshape the job and resume embeddings to get cosine similarity scores
job_embedding = job_embedding.reshape(job_embedding.shape[0], -1)
resume_embeddings = np.concatenate(resume_embeddings, axis=0)
resume_embeddings = resume_embeddings.reshape(resume_embeddings.shape[0], -1)

similarity_scores = cosine_similarity(job_embedding, resume_embeddings)

In [144]:
# Save cosine similarity to the dataset
df['Similarity_Percent'] = [round(i.item()*100, 2) for i in similarity_scores[0]]

In [145]:
df.head()

,Category,Resume,cleaned_resume,Similarity_Percent
0,Testing,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,97.36
1,Testing,Curtis Samuel (123) 456-7890 csamuel@example.c...,curtis samuel csamuel street san diego ca prof...,95.39
2,Testing,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,97.66
3,Testing,"SOHA VANKUDRE\nBoston, MA | Ph: +1(857)-381-54...",soha vankudre boston ph linkedin education mas...,95.68
4,Testing,Venkata Bhargavi S\nBoston MA • (339) 224-1508...,venkata bhargavi boston http portfolio educati...,94.63


In [146]:
# Save the sorted dataframe based on similarity
ds_sorted = df.sort_values(by='Similarity_Percent', ascending=False)

In [147]:
# Rank the similarity based on highest similarity
ds_sorted['Predicted_Rank'] = range(1, len(ds_sorted) + 1)

In [148]:
ds_sorted.head()

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank
11,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency basic powerpoint word out...,98.29,1
17,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency basic powerpoint word out...,98.29,2
16,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,97.85,3
15,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,97.85,4
2,Testing,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,97.66,5


In [149]:
# Manual ranking of the resumes which best fit the job description
# You cn manually list the ranking based on your convenience

manual_rank = [i for i in range(1, len(ds_sorted) + 1)]

In [150]:
# Shuffle the list
random.shuffle(manual_rank)

In [151]:
ds_sorted['Manual_Rank'] = manual_rank
ds_sorted

,Category,Resume,cleaned_resume,Similarity_Percent,Predicted_Rank,Manual_Rank
11,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency basic powerpoint word out...,98.29,1,7
17,Testing,COMPUTER PROFICIENCY â¢ Basic: MS-Office (Pow...,computer proficiency basic powerpoint word out...,98.29,2,10
16,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,97.85,3,14
15,Testing,Skill Set OS Windows XP/7/8/8.1/10 Database MY...,skill set o window database mysql sql server l...,97.85,4,18
2,Testing,SWATHI RUCHIRA\nView Portfolio mswathi.ruchira...,swathi ruchira view portfolio education mp dig...,97.66,5,11
0,Testing,Eleanor Mccartney\n\n4522 Davisson Street\nRus...,eleanor mccartney davisson street rushville em...,97.36,6,3
6,Testing,"Nikitha Kanaparthi\nBoston, MA. 02120 • +1 (20...",nikitha kanaparthi boston linkedin github educ...,96.94,7,16
7,Testing,VIGNAN VENNAMPALLY vennampally.v@northeastern....,vignan vennampally linkedin github education n...,96.65,8,5
8,Testing,Hani Haider\nPhone: 781-491-3977 | Email: haid...,hani haider phone email boston linkedin github...,96.47,9,8
13,Testing,â Willingness to accept the challenges. â ...,willingness accept challenge positive thinking...,96.39,10,2


In [152]:
rbo.RankingSimilarity(ds_sorted['Manual_Rank'].tolist(), ds_sorted['Predicted_Rank'].tolist()).rbo()

0.48660969233724655